In [1]:
import pandas as pd

import os

os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"



import pandas as pd

# Load and combine datasets (as before)
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}
combined_df = pd.concat([datasets['benign'], datasets['dns'], datasets['mitm']], ignore_index=True)
combined_df = combined_df.drop('dst_mac', axis=1)

print(combined_df.shape)

C:\Users\nazih\AppData\Local\Temp\ipykernel_10880\1938280762.py:18: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_10880\1938280762.py:19: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


(883525, 135)


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

def is_hex_mac(mac):
    if pd.isna(mac) or mac == 'unknown':
        return False
    pattern = r'^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$'
    return bool(re.match(pattern, str(mac)))


combined_df = combined_df[~combined_df['src_mac'].apply(is_hex_mac)]

# Drop duplicates
print(f"Number of duplicate rows: {combined_df.duplicated().sum()}")
combined_df = combined_df.drop_duplicates()
print(f"New shape after removing duplicates: {combined_df.shape}")

# Identify categorical columns
categorical_cols = [col for col in combined_df.columns if combined_df[col].nunique() < 10 or combined_df[col].dtype == 'object']
print(f"Categorical features: {categorical_cols}")

for col in categorical_cols:
    combined_df[col] = combined_df[col].astype(str).fillna('unknown')

numerical_cols = combined_df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    combined_df[col] = combined_df[col].fillna(combined_df[col].median())

X = combined_df.drop(['Label', 'src_mac'], axis=1, errors='ignore')
y_attack = combined_df['Label']
y_device = combined_df['src_mac']

# Encode device labels
le_device = LabelEncoder()
y_device_encoded = le_device.fit_transform(y_device)

from sklearn.preprocessing import MultiLabelBinarizer


combined_df['All_Labels'] = combined_df.apply(
    lambda row: [row['Label'], row['src_mac']], axis=1
)

mlb = MultiLabelBinarizer()
Y_multilabel = mlb.fit_transform(combined_df['All_Labels'])

multilabel_class_names = mlb.classes_


X_numeric = X.select_dtypes(include=['number'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)




X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled,       
    Y_multilabel,       
    test_size=0.2, 
    random_state=42
)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)



Number of duplicate rows: 0
New shape after removing duplicates: (432555, 135)
Categorical features: ['src_mac', 'src_ip', 'dst_ip', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'handshake_version', 'handshake_ciphersuites', 'tls_server', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size', 'Label']


C:\Users\nazih\AppData\Local\Temp\ipykernel_10880\734139780.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['All_Labels'] = combined_df.apply(


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier # New Model
from sklearn.metrics import accuracy_score, jaccard_score, classification_report
from sklearn.base import clone

# --- RFE Setup and Execution (Same as before) ---

# RFE Estimator: Use a fast XGBClassifier for feature ranking
xgb_rfe_estimator = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    random_state=42, 
    n_jobs=-1, 
    eval_metric='logloss'
)

TARGET_FEATURES = 50 
rfe_selector = RFE(
    estimator=xgb_rfe_estimator, 
    n_features_to_select=TARGET_FEATURES, 
    step=5,
    verbose=1
)

Y_single_label = Y_train[:, 0]
try:
    multilabel_class_names = mlb.classes_
except NameError:
    multilabel_class_names = [f"Label_{i+1}" for i in range(Y_train.shape[1])]


print(f"Starting RFE to select {TARGET_FEATURES} features...")
rfe_selector.fit(X_train, Y_single_label)

selected_features = X_train.columns[rfe_selector.support_]

X_train_RFE = X_train[selected_features]
X_test_RFE = X_test[selected_features]

print(f"RFE complete. Using {len(selected_features)} features.")
print(f"Selected features: {selected_features.tolist()[:5]}... (first 5 shown)")


# --- Training Multi-Label Random Forest (Binary Relevance) on RFE Features ---

print("\n" + "="*70)
print("--- Training Multi-Label Random Forest (Binary Relevance) on RFE Features ---")
print("="*70)

# Base Random Forest Estimator
rf_base = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    random_state=42,
    n_jobs=-1, # Use all cores for fitting
    class_weight='balanced' # Equivalent to scale_pos_weight in XGBoost
)

num_outputs = Y_train.shape[1]
fitted_rf_estimators = [] 

print("Fitting individual Random Forest models (one per label)...")

for i in range(num_outputs):
    y_col = Y_train[:, i]
    
    # We use class_weight='balanced' inside the base estimator, so no manual
    # weight calculation (like scale_pos_weight) is strictly necessary here.
    rf_estimator = clone(rf_base) 
    
    # Train the estimator for the current label
    rf_estimator.fit(X_train_RFE, y_col)
    
    fitted_rf_estimators.append(rf_estimator)

# --- Prediction ---

Y_pred_rf = np.zeros_like(Y_test)

for i, estimator in enumerate(fitted_rf_estimators):
    # Get the final prediction for the current label on the test set
    Y_pred_rf[:, i] = estimator.predict(X_test_RFE)


# --- Evaluation ---

subset_acc = accuracy_score(Y_test, Y_pred_rf)
# Use 'samples' average for Jaccard, which is typically desired in multi-label classification
jaccard = jaccard_score(Y_test, Y_pred_rf, average='samples', zero_division=0)

print("\n" + "="*70)
print("FINAL RANDOM FOREST (RFE FEATURES) TEST SET RESULTS")
print("="*70)
print(f"Subset Accuracy (Exact Match): {subset_acc:.4f}")
print(f"Jaccard Score (Label Similarity): {jaccard:.4f}")

print("\n" + "="*70)
print("DETAILED MULTI-LABEL CLASSIFICATION REPORT (Per-Label Metrics)")
print("="*70)

report = classification_report(
    Y_test, 
    Y_pred_rf, 
    target_names=multilabel_class_names, 
    zero_division=0,
    output_dict=False
)
print(report)

Starting RFE to select 50 features...
Fitting estimator with 107 features.


c:\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:17:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting estimator with 102 features.


c:\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:17:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting estimator with 97 features.


...

Fitting estimator with 57 features.


c:\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting estimator with 52 features.


c:\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:19:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


RFE complete. Using 50 features.
Selected features: [0, 1, 2, 3, 6]... (first 5 shown)

--- Training Multi-Label Random Forest (Binary Relevance) on RFE Features ---
Fitting individual Random Forest models (one per label)...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression # New Model
from sklearn.svm import LinearSVC # New Model (for SVM with linear kernel)
from sklearn.metrics import accuracy_score, jaccard_score, classification_report
from sklearn.base import clone
from sklearn.preprocessing import MultiLabelBinarizer
import catboost as cb # CatBoost Added
import lightgbm as lgb # LightGBM Added

# --- RFE Setup and Execution (Assuming it has been run successfully) ---
# NOTE: The RFE selection is kept the same for a fair feature set.

# --- Mock Data and RFE Run (for standalone execution) ---
Y_single_label = Y_train[:, 0]
try:
    multilabel_class_names = mlb.classes_
except NameError:
    multilabel_class_names = [f"Label_{i+1}" for i in range(Y_train.shape[1])]


TARGET_FEATURES = 50
xgb_rfe_estimator = XGBClassifier(n_estimators=100, max_depth=5, random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss')
rfe_selector = RFE(estimator=xgb_rfe_estimator, n_features_to_select=TARGET_FEATURES, step=5, verbose=0)

# Run RFE
print(f"Starting RFE to select {TARGET_FEATURES} features...")
rfe_selector.fit(X_train, Y_single_label)

selected_features = X_train.columns[rfe_selector.support_]
X_train_RFE = X_train[selected_features]
X_test_RFE = X_test[selected_features]

print(f"RFE complete. Using {len(selected_features)} features.")

model_configs = {
    # Traditional Models
    'RandomForest': {
        'base_estimator': RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1, class_weight='balanced'),
        'predictions': None
    },
    'LogisticRegression': {
        'base_estimator': LogisticRegression(solver='liblinear', random_state=42, n_jobs=1, class_weight='balanced'),
        'predictions': None
    },
    'LinearSVC': {
        'base_estimator': LinearSVC(random_state=42, class_weight='balanced', dual=True),
        'predictions': None
    },
    'AdaBoost': {
        # AdaBoost often uses a shallow tree (DecisionTreeClassifier) as its base
        'base_estimator': AdaBoostClassifier(n_estimators=500, random_state=42),
        'predictions': None
    },
    # State-of-the-Art Boosting Models
    'LightGBM': {
        'base_estimator': lgb.LGBMClassifier(
            n_estimators=500, 
            random_state=42, 
            n_jobs=-1,
            # Handle imbalance via class weights
            scale_pos_weight=1, # This will be set dynamically in the loop
            objective='binary' 
        ),
        'predictions': None
    },
    'CatBoost': {
        'base_estimator': cb.CatBoostClassifier(
            iterations=500, 
            random_state=42, 
            verbose=0, 
            thread_count=-1,
            # Handle imbalance via class weights
            auto_class_weights='Balanced', 
            loss_function='Logloss'
        ),
        'predictions': None
    }
}

num_outputs = Y_train.shape[1]
results = {}

print("\n" + "="*70)
print("--- Starting Multi-Label Model Comparison (Binary Relevance) ---")
print("="*70)

# --- Training and Prediction Loop for all models ---
for model_name, config in model_configs.items():
    print(f"\n--- Training {model_name} ---")
    
    base_estimator = config['base_estimator']
    fitted_estimators = [] 

    for i in range(num_outputs):
        y_col = Y_train[:, i]
        
        # Calculate weight for highly imbalanced models (XGBoost, LightGBM)
        pos_count = np.sum(y_col == 1)
        neg_count = np.sum(y_col == 0)
        weight = neg_count / pos_count if pos_count > 0 else 1.0

        estimator = clone(base_estimator) 
        
        # Custom setting for models that use scale_pos_weight
        if model_name in ['LightGBM']:
            estimator.set_params(scale_pos_weight=weight)
        
        # Fit the model
        # CatBoost requires its own data format if not using the default Scikit-learn wrapper
        # We will use the standard wrapper for simplicity, as auto_class_weights is set.
        estimator.fit(X_train_RFE, y_col)
        fitted_estimators.append(estimator)

    # Predict
    Y_pred = np.zeros_like(Y_test)
    for i, estimator in enumerate(fitted_estimators):
        Y_pred[:, i] = estimator.predict(X_test_RFE)
    
    model_configs[model_name]['predictions'] = Y_pred

    # Evaluate
    subset_acc = accuracy_score(Y_test, Y_pred)
    jaccard = jaccard_score(Y_test, Y_pred, average='samples', zero_division=0)
    
    results[model_name] = {'subset_accuracy': subset_acc, 'jaccard_score': jaccard}
    
    print(f"[{model_name}] Subset Accuracy: {subset_acc:.4f}")
    print(f"[{model_name}] Jaccard Score: {jaccard:.4f}")

# --- Final Summary and Detailed Report ---

print("\n" + "="*70)
print("COMPREHENSIVE MULTI-LABEL MODEL SUMMARY")
print("="*70)

# Print Summary Table
print(f"{'Model':<20} | {'Subset Accuracy':<18} | {'Jaccard Score':<15}")
print("-" * 70)
# NOTE: You should manually insert your original XGBoost results here to complete the comparison!
for model_name, metrics in results.items():
    print(f"{model_name:<20} | {metrics['subset_accuracy']:.4f}{'':<17} | {metrics['jaccard_score']:.4f}")
